In [2]:
import cobra.test
import os
from os.path import join
from cobra.io import read_sbml_model
model = cobra.io.load_json_model(join("RehMBEL1391_sbml_L3V1.json"))
from cobra import Model, Reaction, Metabolite

# Adding D-glucose to the medium

The model we are using has D-fructorse as carbon source.

## Standard medium content of the model

In [3]:
model.medium   #EX_fru_e is D-frucose

{'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_pi_e': 100.0,
 'EX_cobalt2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_k_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_fru_e': 3.0,
 'EX_nh4_e': 10.0,
 'EX_na_e': 10.0,
 'EX_o2_e': 18.5,
 'EX_mobd_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 100.0}

We will be trying to include D-glucose in the medium and look at the bimass and production of PHB.

In [ ]:
With the current medium the biomass, yeald of phb and 

In [62]:
Objective=model.objective
solution=model.optimize().objective_value
print(Objective)
print('The models maximum biomass productivity:', solution, '/h')

Maximize
1.0*Biomass - 1.0*Biomass_reverse_57a34
The models maximum productivity: 0.2852858664276785 mmol/gDW*h


In [ ]:
with model:
    model.objective = model.reactions.EX_pbhb_e
    print(model.slim_optimize())

In [25]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 0
    medium
    model.medium=medium
    print(model.objective, model.optimize)
    model.objective = model.reactions.EX_pbhb_e
    print(model.slim_optimize())
    model.medium = medium
    print(medium)
    print(model.slim_optimize())

Maximize
1.0*Biomass - 1.0*Biomass_reverse_57a34 <bound method Model.optimize of <Model RehMBEL1391 at 0x2add8e1c2e0>>
0.0
{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0}
0.0


Then we need to add a new reaction and the D-glucose at matobilite 

In [30]:
reaction = Reaction('D-GLC')
reaction.name = 'D-glucose transport via PEP:Pyr PTS'
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [31]:
EX_glc_D_e = Metabolite(
    'EX_glc_D_e',
    formula='C6H12O6',
    name='D-glucose',
    compartment='c')

In [32]:
reaction.add_metabolites({
    EX_glc_D_e: -1.0,
    model.metabolites.pep_c: -1.0,
    model.metabolites.pyr_c: 1.0,
    model.metabolites.g6p_c: 1.0
})
reaction.reaction  # This gives a string representation of the reaction

'EX_glc_D_e + pep_c --> g6p_c + pyr_c'

In [33]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 0
    medium['EX_succ_e'] = 10
    print(medium)
    model.medium=medium
    model.objective = model.reactions.EX_pbhb_e
    print(model.slim_optimize())
    model.medium = medium
    #print(medium)
    #print(model.slim_optimize())
    

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_succ_e': 10}
6.45454545454547


In [46]:
model.reactions.get_by_id('EX_succ_e')

Reaction identifier,EX_succ_e
Name,EX_succ_e
Memory address,0x02adddfb4d60
Stoichiometry,succ_e --> Succinate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [55]:
for metabolites in model.metabolites.query('succ','name'):
    print(metabolites.name)

Naphthyl-2-methyl-succinic acid
Naphthyl-2-methyl-succinyl-CoA
Naphthyl-2-oxomethyl-succinyl-CoA
1-(5'-Phosphoribosyl)-5-amino-4-(N-succinocarboxamide)-imidazole
Naphthyl-2-hydroxymethyl-succinyl CoA
Naphthyl-2-methylene-succinyl-CoA
N-(L-Arginino)succinate


In [53]:
model.reactions.query('succ','name')

[<Reaction EX_succ_e at 0x2adddfb4d60>,
 <Reaction TARTRt7 at 0x2adde6fcf40>,
 <Reaction HYDRTS at 0x2adde82f610>,
 <Reaction NAPMSCD at 0x2adde974a30>,
 <Reaction SDPDS at 0x2addea22610>,
 <Reaction THDPS at 0x2addea54af0>,
 <Reaction ARGSL at 0x2addec03f40>,
 <Reaction CITt7 at 0x2addec19df0>,
 <Reaction SCCNAPS at 0x2addec67d30>,
 <Reaction ARGSS at 0x2adde7aecd0>,
 <Reaction PRASUCS at 0x2addef5dfd0>,
 <Reaction ADSL1r at 0x2addf0f1c70>,
 <Reaction ADSUCL1 at 0x2addf0f12e0>,
 <Reaction OGDH2 at 0x2addf233e20>,
 <Reaction ADSS at 0x2addf312be0>]

In [52]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 0
    #medium['EX_glc_D_e'] = 0
    medium['EX_succ_e'] = 0
    model.medium =medium
    print(medium)
    print(model.slim_optimize())
    #model.medium = medium
    #print(medium)
    #print(model.slim_optimize())

{'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_pi_e': 100.0, 'EX_cobalt2_e': 10.0, 'EX_cl_e': 10.0, 'EX_k_e': 10.0, 'EX_fe3_e': 10.0, 'EX_so4_e': 10.0, 'EX_fru_e': 0, 'EX_nh4_e': 10.0, 'EX_na_e': 10.0, 'EX_o2_e': 18.5, 'EX_mobd_e': 10.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 100.0, 'EX_succ_e': 0}
0.0


In [ ]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 10
    print(medium)
    model.medium=medium
    model.objective = model.reactions.EX_pbhb_e
    print(model.slim_optimize())

In [ ]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 10
    medium['EX_o2_e'] =0
    print(medium)
    model.medium=medium
    model.objective = model.reactions.EX_pbhb_e
    solution=model.optimize()
    print(solution)

In [ ]:
with model:
    medium = model.medium
    #medium['EX_fru_e'] = 10
    medium['EX_o2_e'] =0
    print(medium)
    model.medium=medium
    model.objective = model.reactions.EX_pbhb_e
    solution=model.optimize()
    print(solution.fluxes[solution.fluxes != 0.])
    print(solution)

In [67]:
for reactions in model.metabolites.get_by_id('phb_c').reactions:
    print(reactions)

PHAS: r3hbcoa_c <=> coa_c + phb_c
PHBt: h_c + phb_c --> h_e + phb_e
PHBD: h2o_c + phb_c <=> h_c + r3hbn2_c + r3rhbb_c


In [ ]:
###########See if it makes more of phb

In [27]:
model.objective = model.reactions.EX_pbhb_e
print(model.objective)

Maximize
1.0*EX_pbhb_e - 1.0*EX_pbhb_e_reverse_f6e04


In [28]:
solution = model.optimize()
print(solution)
solution.fluxes

<Solution 3.763 at 0x1fed6b9a0a0>


EX_gln__L_e       0.0
EX_galctn__D_e    0.0
EX_c140_e         0.0
EX_15dap_e        0.0
EX_3hbenzot_e     0.0
                 ... 
THZPSN            0.0
MTHF3ES_c         0.0
HYDS              0.0
EX_h2_e           0.0
H2td              0.0
Name: fluxes, Length: 1538, dtype: float64

In [ ]:
with model:
    medium = model.medium
    medium['EX_fru_e'] = 0
    medium['EX__e'] = 10
    print(medium)
    print(model.slim_optimize())
    #model.medium = medium
    #print(medium)
    #print(model.slim_optimize())

In [84]:
medium = model.medium
with model:
    medium['EX_fru_e'] = 10
    medium['EX_succ_e'] = 0
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_BIOMASS_c'])

0.9304279805448478


In [39]:
medium = model.medium
with model:
    medium['EX_fru_e'] = 0
    medium['EX_succ_e'] = 10
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_BIOMASS_c'])

0.5343325098873601
